In [2]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers import *

Using TensorFlow backend.


In [3]:
input_tensor = Input((400, 80, 1))
x = input_tensor
for i in range(3):
    x = Conv2D(32 * 2 ** i, (3, 3), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32 * 2 ** i, (3, 3), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

In [5]:
x.get_shape().as_list()

[None, 46, 6, 128]